# STXAE - Advanced structure tensor fiber analysis as basis for element wise orientation mapping onto finite elements
*Author: Niels Jeppesen (<niejep@dtu.dk>)*, Robert Auenhammer (<robaue@chalmers.se>)

In this notebook we use Structure Tensor (ST) for orientation analysis of glass fiber composites. Those orientations are then mapped element-wise onto a finite element mesh. All types of elements, shells and solids, with all orders are possible. This notebook integrates the Structure Tensor method in an X-ray computer tomography aided engineering (XAE) process. 

The ```structure-tensor``` package we will be using here is a 2D and 3D strcture tensor package for Python implemented by [Vedrana A. Dahl](mailto:vand@dtu.dk) and [Niels Jeppesen](mailto:niejep@dtu.dk).

*****************************

## Installation
To run this notebook there are some prerequisites that must be installed in our Python environment. We generally recommend [creating a new Python environment](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html) to run the notebooks in, for instance using ```conda create -n <env_name> python=3.7```. This way we avoid conflicts with packages in your existing environment.

### Option 1
Install dependencies (for this notebook) using ```pip install numpy scipy scikit-image matplotlib nibabel tqdm structure-tensor```. To start the notebook we will of course also need ```jupyter```.

### Option 2
Run ```pip install -r requirements.txt``` using the included ```requirements.txt``` file. **This will install specific versions of all packages needed to run this notebook, overwriting any current versions of those packages in our environment.** This also includes packages used in our other notebook.

Now, let's go ahead and import the required modules. The ```structure_tensor_workers.py``` file should be in the notebook folder. 

In [1]:
import os
from multiprocessing import Pool

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from scipy import ndimage
from scipy.spatial.transform import Rotation as R
from skimage import filters
from structure_tensor import eig_special_3d, structure_tensor_3d
from tqdm import tqdm

from structure_tensor_workers import calculate_angles, get_crops, init_worker, structure_tensor_analysis_v1

import pandas as pd
import csv
import numpy.f2py

## Load and display data
First, we'll load a sample of the data and some meta data. We will be using the following folder structure:
- ```../notebooks``` contains the notebooks.
- ```../originals``` should contain the original data files.
- ```../tmp``` should contain any temperary files or output we create.
- ```../notebooks/figures/<file_name>``` contains optionally saved figures.

In [2]:
# Set file name and path.
file_name = 'HF401TT-13_FoV16.5_Stitch.txm_binning8.nii'
file_path = os.path.join('../data/', file_name)

In [3]:
# Change this to True to save figures to folder.
save_figures = False

# Create folder for to save figures.
fig_path = os.path.join('figures', os.path.basename(file_name))
if save_figures and fig_path and not os.path.exists(fig_path):
    os.makedirs(fig_path)

In [4]:
# Load data.
nii_file = nib.load(file_path)

# Read meta data.
data_shape = nii_file.shape
data_type = nii_file.get_data_dtype()
voxel_size = nii_file.affine[0, 0]
# in micrometer
fiber_diameter = 17

print('Shape:', data_shape)
print('Data type:', data_type)
print('Voxel size:', voxel_size, 'μm')
print('Fiber diameter size:', fiber_diameter, 'μm')

Shape: (74, 240, 528)
Data type: uint16
Voxel size: 64.62985229492188 μm
Fiber diameter size: 17 μm


   
Let's have a need look at the data, to ensure it was loaded correctly.

In [5]:
fig, axs = plt.subplots(1, 2, figsize=(20, 20))
i = data_shape[-1] // 3
img = nii_file.dataobj[..., i]
axs[0].imshow(img, cmap='gray')
i = data_shape[-1] // 3 * 2
img = nii_file.dataobj[..., i]
axs[1].imshow(img.squeeze(), cmap='gray')
plt.show()

## Crop data
Some of the data may not of interest to us, so let's crop out the area of interest. You can cut out any part of the data you want to analyze.

In [6]:
start_X=0
end_X=nii_file.shape[0]
start_Y=0
end_Y=nii_file.shape[1]
start_Z=0
end_Z=nii_file.shape[2]
x_slice = slice(start_Z, end_Z)
y_slice = slice(start_Y, end_Y)
z_slice = slice(start_X, end_X)

Let's have a look at the cropped area. The TXM reader can be a bit slow, so we will only look at YZ-slices for now. We may have to come back and adjust the cropping on the X-axis later.

In [7]:
fig, axs = plt.subplots(1, 2, figsize=(20, 20))
i = data_shape[-1] // 3 * 2
img = nii_file.dataobj[z_slice, y_slice, i]
axs[0].imshow(img, cmap='gray')
i = data_shape[-1] // 3
img = nii_file.dataobj[z_slice, y_slice, i]
axs[1].imshow(img, cmap='gray')
plt.show()

Create new shape. **Note: We rotate 90 degrees around the X axes to match convention.**

In [8]:
data_shape = (x_slice.stop - x_slice.start, y_slice.stop - y_slice.start, z_slice.stop - z_slice.start)
print('New shape:', data_shape)

New shape: (528, 240, 74)


## Save as RAW
To work with the data, we will save it as a raw data file, which will allow us to access the data using ```numpy.memmap```. Using a memory-mapped file is both convenient and fast, especially since we will be accessing the data many times from different processes.

We will be placing all our processed data in a temporary folder, as it can be re-created using this notebook if deleted. We will also use data in the temp folder as a cache, so we don't have to create the RAW volume if it already exists. Thus, if you change the volume cropping etc., be sure to delete the RAW volume from the temp folder and create a new one.

In [9]:
temp_folder = '../tmp/'
if temp_folder and not os.path.exists(temp_folder):
    os.mkdir(temp_folder)

In [10]:
data_path = os.path.join(temp_folder, file_name + f'_{x_slice.start}-{x_slice.stop}_{y_slice.start}-{y_slice.stop}_{z_slice.start}-{z_slice.stop}.raw')
if not os.path.exists(data_path):
    
    # Number of images read at a time.
    # Reduce this to lower memory usage.
    chunck_size = 1024
    
    print('Creating new file:', data_path)
    data = np.memmap(data_path, dtype=data_type, shape=data_shape, mode='w+')
    
    for i in tqdm(range(0, data_shape[0], chunck_size)):
        x0 = i + x_slice.start
        x1 = min(x_slice.stop, x0 + chunck_size)
        
        chunck = nii_file.dataobj[z_slice, y_slice, x0:x1]
        data[i:i + chunck_size] = np.rot90(chunck, k=3, axes=(0, 2))
        

else:
    print('Using existing file:', data_path)
data = np.memmap(data_path, dtype=data_type, shape=data_shape, mode='r')

100%|██████████| 1/1 [00:00<00:00, 34.45it/s]

Creating new file: ../tmp/HF401TT-13_FoV16.5_Stitch.txm_binning8.nii_0-528_0-240_0-74.raw


Let's have a look some slices from all three axes.

**Important: Remember to look at figures below to verify that the correct data is loaded into the ```data``` variable. If you've changed the slicing or the data you're working with, you may want to delete the RAW data cached in the ```temp_folder``` folder.**

In [11]:
plt.figure(figsize=(20, 20))
ax = plt.subplot(2, 2, 1)
ax.imshow(data[data.shape[0] // 3], cmap='gray')
ax = plt.subplot(2, 2, 2)
ax.imshow(data[data.shape[0] // 3 * 2], cmap='gray')
ax = plt.subplot(2, 2, 3)
ax.imshow(data[:, data.shape[1] // 2], cmap='gray')
ax = plt.subplot(2, 2, 4)
ax.imshow(data[..., data.shape[-1] // 2], cmap='gray')
plt.show()

## Choosing fiber threshold
To choose a good intensity threshold for the fibers we can use a histogram and choose the value manually. Alternatively we can use a method such as Otsu's threshold. For choosing the threshold we make sure only to include inside-sample data. This makes it easier to separate foreground (fibers) and background inside the sample.

In [12]:
%%time
threshold_bins = 512
threshold_data = data[25:-25, 25:-25, 25]

CPU times: user 38 µs, sys: 4 µs, total: 42 µs
Wall time: 31.5 µs


In [13]:
otsu_threshold = filters.threshold_otsu(threshold_data.reshape(-1, 1), nbins=threshold_bins)
hand_picked_threshold = 41000

In [14]:
#This plot can be skipped if time is important
ax = plt.subplot(1, 1, 1)
ax.hist(threshold_data.flat, bins=threshold_bins, range=[5000, 60000], color="grey",
    edgecolor="1",
    linewidth=0.2)
ax.set_xlabel('Greyscale value', fontsize=15, fontweight ='bold')
ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)
ax.axvline(otsu_threshold, c='r')
ax.axvline(hand_picked_threshold, c='g')
plt.show()
print('Otsu threshold:', otsu_threshold)
print('Hand-picked threshold:', hand_picked_threshold)

Otsu threshold: 38164
Hand-picked threshold: 41000


In [15]:
print('Otsu fiber fraction:', round(np.count_nonzero(threshold_data >= otsu_threshold) / threshold_data.size, 3))
print('Hand-picked fiber fraction:', round(np.count_nonzero(threshold_data >= hand_picked_threshold) / threshold_data.size, 3))

Otsu fiber fraction: 0.723
Hand-picked fiber fraction: 0.625


## Structure tensor
We set $\sigma$ and $\rho$ based on the size of the fibers that we want to analyze. $\sigma$ and $\rho$ are the only two parameters that need to be defined for this material orientation analysis. The value  For more details see the related paper and *StructureTensorFiberAnalysisDemo* notebook. 

Then, we set the block size (```crop_size```), which will determine the maximum size of the blocks we will partition the data into for the ST calculation. The maximum black size will depend on ```crop_size```, the ```truncate``` value used for Gaussian filters (default is 4), and $\rho$ or $\sigma$ (usually $\rho$ because ti is largest).

We also set the ```fiber_threshold``` and specify a list of devices to use for the calculation. The list determines how the blocks will be distributed between devices for calculations. If we have a dual GPU system supporting CUDA, we can specify ```['cuda:0', 'cuda:1']```, which will distribute blocks evenly between the two GPUs. ```['cuda:0', 'cuda:1', 'cuda:1']``` would move two thirds to GPU 1, while ```['cuda:0', 'cuda:1', 'cpu']``` will move one third of the blocks to the CPU. **Remember to update the ```device``` list to match your system resources. Specifying CUDA devices that are not available will result in exceptions and/or undefined behaviour.** If you don't have CUDA GPUs available, just set ```device = ['cpu']``` and specify the number of processes later.

The ```class_vectors``` area used to segment the voxels based on the orientations of the ST eigenvectors. It is a list of unit vectors, which represent each of the fiber classes. The first vector is considered the primary orientation and will be used for calculating orientation metrics for each voxel. For more details see the related paper and *StructureTensorFiberAnalysisDemo* notebook.

Lastly, ```bins_per_degree``` determines the number of bins used per degree when aggregating the orientation metrics. 

In [16]:
r = fiber_diameter / 2 / voxel_size
sigma = round(np.sqrt(r**2 / 2), 2)
rho = 4 * sigma
sigma=0.6
rho=1.5

print('sigma:', sigma)
print('rho:', rho)

crop_size = 334
print('crop_size:', crop_size)
print('Maximum block size:', crop_size + int(4 * max(rho, sigma) + 0.5))

fiber_threshold = 41000
# Important: Listing invalid CUDA devices may results in exceptions.
#device = ['cuda'] 
device = ['cpu']

class_names = ['0', '45', '-45', '90']
class_vectors = np.array([[0, 0, 1], [0, 1, 1], [0, -1, 1], [0, 1, 0]], dtype=np.float64)
class_vectors /= np.linalg.norm(class_vectors, axis=-1)[..., np.newaxis]

bins_per_degree = 4

sigma: 0.6
rho: 1.5
crop_size: 334
Maximum block size: 340


We can use the ```get_crops``` function to partition the data into blocks (```crops```) with proper padding. However, we will actually only use ```len(crops)``` here, as we will use ```multiprocessing``` to distribute the blocks accross multiple devices. We may include a function just for calculating the number of blocks/crops at a later point.

In [17]:
# Get crops as memory views.
crops, crop_positions, crop_paddings = get_crops(data, max(sigma, rho), crop_size=crop_size)

We will be using the ```structure_tensor_analysis_v1``` function to calculate the structure tensor, $S$, and do the eigendecomposition for the blocks in parallel. We will be saving the following metrics to the disk using memory maps created below:

1. ```vec_class``` contains a label for each voxel. 
2. ```eta_os``` contains a stiffness estimate, $\eta_O$, for each voxel
3. ```theta``` contains the angle, $\theta$ between ```vec``` and class 0, at each voxel.
4. ```out_xy_angle``` angle between```vec``` and the XY-plane (out-of-plane orientation).
5. ```in_xy_angle``` rotation of ```vec``` in the XY-plane (in-plane orientation).

To calcualte these metrics, the ```structure_tensor_analysis_v1``` uses the ```calculate_angles``` function, which is explained further in the *StructureTensorFiberAnalysisDemo* notebook. However, the metrics returned by ```structure_tensor_analysis_v1``` have been aggregated (binned), as returning all the metrics for each block unaggregated would obvously consume large amounts of memory and be infeasible for large volumes. We will combine the aggregated data from each block afterwards.

In the code below we create memory-mapped files for the eigenvectors, along with three other metrics. The ```structure_tensor_analysis_v1``` function will be using these to save the metrics for the volume straight to the disk, which may require a significant amount of disk space, but shouldn't result in memory issues. Besides the five types shown below, the function also supports saving $S$ and the eigenvalues. See ```structure_tensor_workers.py``` for details. In the example below we will be saving the results using 16-bit precision data types to save space. This is usually fine for visualization and most statistics. Saving the metrics to disk is optional. **If you don't need the per voxel orientations for later, don't create the memory-maps and remove the entries from the ```init_args``` dictionary below.** This will save you a lot of disk space and probably reduce the processing time.

In [18]:
# Output names, dtypes and shapes.
map_names = ['vec', 'vec_class', 'theta', 'in_xy_angle', 'out_xy_angle']
map_dtypes = [np.float16, np.uint8, np.float16, np.float16, np.float16]
map_shapes = [(3, ) + data.shape, data.shape, data.shape, data.shape, data.shape]

# Output paths.
map_paths = {n: data_path.replace('.raw', f'-{n}-{sigma}-{rho}-{fiber_threshold}.raw') for n in map_names}
map_paths['vec'] = data_path.replace('.raw', f'-vec-{sigma}-{rho}.raw')

# Create maps.
maps = {}
new_maps = {}

for n, dtype, shape in zip(map_names, map_dtypes, map_shapes):
    path = map_paths[n]
    create_map = not os.path.exists(path) or os.stat(path).st_size != np.product(shape) * np.dtype(dtype).itemsize
    
    if create_map:
        print(f'Creating memory mapped file:\n{path}')
        mmap = np.memmap(path, dtype=dtype, shape=shape, mode='w+')
        if np.issubdtype(dtype, np.floating) and n not in ['S', 'val', 'vec']:
            mmap[:] = np.nan
            
        new_maps[n] = (path, dtype)
        
    maps[n] = np.memmap(path, dtype=dtype, shape=shape, mode='r')

Creating memory mapped file:
../tmp/HF401TT-13_FoV16.5_Stitch.txm_binning8.nii_0-528_0-240_0-74-vec-0.6-1.5.raw
Creating memory mapped file:
../tmp/HF401TT-13_FoV16.5_Stitch.txm_binning8.nii_0-528_0-240_0-74-vec_class-0.6-1.5-41000.raw
Creating memory mapped file:
../tmp/HF401TT-13_FoV16.5_Stitch.txm_binning8.nii_0-528_0-240_0-74-theta-0.6-1.5-41000.raw
Creating memory mapped file:
../tmp/HF401TT-13_FoV16.5_Stitch.txm_binning8.nii_0-528_0-240_0-74-in_xy_angle-0.6-1.5-41000.raw
Creating memory mapped file:
../tmp/HF401TT-13_FoV16.5_Stitch.txm_binning8.nii_0-528_0-240_0-74-out_xy_angle-0.6-1.5-41000.raw


In [19]:
# Get memory maps.
vec = maps['vec']
vec_class = maps['vec_class']
theta = maps['theta']
in_xy_angle = maps['in_xy_angle']
out_xy_angle = maps['out_xy_angle']

Now we're finally ready to perform the analysis. We will be using ```multiprocessing.Pool``` to distribute the work across multiple CPU cores. If specified in the ```device``` list, the CPU will offload the majority of the work to a GPU, otherwise it'll do the calculations itself. Here, we will create four processes for each device in the list (```processes=4 * len(device)```). As our ```device``` list contains four GPUs, we will be starting 16 processes, four for each GPU. Beware that this may require a significant amount of GPU memory. **If you experience out of memory exceptions, either reduce ```crop_size``` and/or the number of processes per device.** Change the number of processes to fit your system. 

In [20]:
if __name__ ==  '__main__':
    results = []
    
    init_args = {'data': data_path,
                 'dtype': data.dtype,
                 'shape': data.shape,
                 'rho': rho,
                 'sigma': sigma,
                 'crop_size': crop_size,
                 'threshold': fiber_threshold,
                 'class_vectors': class_vectors,
                 'bins_per_degree': bins_per_degree,
                 'device': device
                }
    
    # Add output maps to args.
    for n in new_maps:
        path, dtype = new_maps[n]
        init_args[n] = path
        init_args[f'{n}_dtype'] = dtype

    with Pool(processes=4 * len(device), initializer=init_worker, initargs=(init_args,)) as pool:
        for res in tqdm(pool.imap_unordered(structure_tensor_analysis_v1, range(len(crops)), chunksize=1), total=len(crops)):
            results.append(res)

100%|██████████| 2/2 [00:03<00:00,  1.86s/it]


All shared variables are passed to the workers using the ```init_args``` dictionary and ```init_worker```function. This function is only called once per worker. When called, it will create all the memory-maps needed to read and write data during calculations, based on ```init_args```. We will use the ```pool.imap_unordered``` function to tell the workers to perform the calculations for a block, by passing the ```structure_tensor_analysis_v1``` function along with the index of the block which we want to analyze. The returned results are a tuple of aggregated metrics, as decribed earlier. For more details see the ```structure_tensor_analysis_v1``` code. Below we will show have to combine the aggregated metrics and display them as histograms.

Another option is to save all the metrics to disk (using memory-maps as described earlier) with reasonable precision and perform the statistics directly on the full orientation volumes. This approach would be similar to what's done in the *StructureTensorFiberAnalysisDemo* notebook, except there data is kept in memory instead of on the disk. If you use this approach you may simply ignore the aggregated data returned by ```structure_tensor_analysis_v1```, but it will require you to use a significant amount of disk space to store all the orientation and segmentation data. If the volumes are very big, working with them may also be slow and memory intensive, even if you use ```memmap``` to access the data.

## Validation Structure Tensor outcome
The following cells serve as validation possiblity for the outcome of the calculate structure tensor above. Those cells do not have to be run for the mapping later on.

******

## Histograms
The figures above are great for validating that we've done the calculations correctly and that our results are meaningful. However, even with this many figures we're only actually showing a very small fraction of the volume. To get a proper overview of the fiber orientations and classes we need to aggregate the information in a meaningful way. One way to do this is to use histograms. Since we have saved all the data to the memory maps, we just need to aggregate the data that they contain.

**Note: To load the memory maps from disk later, simply use ```np.memmap```, for example ```theta = np.memmap(path, dtype=dtype, shape=shape, mode='r')```.**

By combining the threshold mask (indicating what is fiber) with the class labels, we can create a fiber class mask for each class.

In [21]:
# Create threshold mask.
threshold_mask = data >= fiber_threshold
# Count number of fiber voxels.
fiber_count = np.count_nonzero(threshold_mask)
# Create a mask for each fiber class.
vec_class_masks = [threshold_mask] + [(vec_class == l) & threshold_mask for l in range(len(class_vectors))]

### Fiber fractions
Using these masks we calculate the fraction of the fibers belonging to each class.

In [22]:
fiber_fractions = [np.count_nonzero(m) / fiber_count for m in vec_class_masks[1:]]
for i, f in enumerate(fiber_fractions):
    c = class_names[i]
    print(f'Fraction ({c}): {round(f, 6)}')

Fraction (0): 0.734655
Fraction (45): 0.109877
Fraction (-45): 0.118585
Fraction (90): 0.036883


In [23]:
plt.bar(class_names, fiber_fractions, align='center', width=0.5, color="grey")
plt.xticks(range(len(fiber_fractions)))
plt.xlabel('Class', fontsize=15, fontweight ='bold')
plt.ylabel('Fraction', fontsize=15, fontweight ='bold')
plt.tick_params(axis='x', labelsize=13)
plt.tick_params(axis='y', labelsize=13)
plt.show()

### Angle from X ($\theta$)
The histograms below show the distribution of the absolute angles between the fibers and the X-axis unit vector, along with the median angle. The resoulution of the histograms is determined by ```bins_per_degree```.

In [24]:
# Four bins per degree = 0.25 degree resolution.
bins_per_degree = 4

In [25]:
def hist(i, ax):
    # Calculate theta for each class using the masks.
    values = theta[vec_class_masks[i]]
    values = np.array(values[~np.isnan(values)], dtype=np.float32)
    
    if i == 0:
        title = 'All classes'
        ax.set_xlim([0, 90])
    else:
        class_name = class_names[i - 1]
        title = f'Class {class_name}'
        class_value = int(class_name)
        ax.set_xlim([max(0, abs(class_value) - 30) , min(90, abs(class_value) + 30)])
    
        m = np.median(values).item()
        ax.axvline(m, c='k', ls='-', label=f'$Med={round(m, 2)}\degree$')
        plt.legend(prop={"size":14})

    ax.set_title(title, fontsize=18)
    ax.hist(values, density=True, bins=np.arange(0, 90 + 1 / bins_per_degree, 1 / bins_per_degree), color="grey",
    edgecolor="1",
    linewidth=0.7)
    ax.set_xlabel('Angle [$\degree$]', fontsize=15, fontweight ='bold')
    ax.set_ylabel('Fraction', fontsize=15, fontweight ='bold')
    ax.tick_params(axis='x', labelsize=13)
    ax.tick_params(axis='y', labelsize=13)

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Angle from X', fontsize=22)

for i in tqdm(range(len(class_names) + 1)):
    ax = plt.subplot(2, 3, i + 1)
    hist(i, ax)
    
plt.tight_layout()
plt.subplots_adjust(top=0.85)  
plt.show()

100%|██████████| 5/5 [00:02<00:00,  2.05it/s]


### In-plane orientation (rotaiton in XY-plane)
The histograms below show the distribution of the rotation around the Z-axis (in-XY-plane rotation), the mean and the standard deviation for each class.

In [26]:
def hist(i, ax):
    values = in_xy_angle[vec_class_masks[i]]
    values = np.array(values[~np.isnan(values)], dtype=np.float32)
    
    if i == 0:
        title = 'All classes'
        class_value = 0
        ax.set_xlim([-90, 90])
        
        ax.axvline(0, c='k', ls='--')
        ax.axvline(-45, c='k', ls='--')
        ax.axvline(45, c='k', ls='--')
    else:
        class_name = class_names[i - 1]
        title = f'Class {class_name}'
        class_value = int(class_name)
        
        ax.set_xlim([class_value - 20, class_value + 20])
#        ax.axvline(class_value, c='k', ls='--', label=f'${class_value}\degree$')

    if class_value == 90:
        ax.set_xticks(range(class_value - 30, class_value + 31, 10))
        ax.set_xticklabels([60, 70, 80, '$\pm90$', -80, -70, -60])
        bins = np.arange(0, 180 + 1 / bins_per_degree, 1 / bins_per_degree)
        values[values < 0] = 90 + (values[values < 0] + 90)
    else:
        bins = np.arange(-90, 90 + 1 / bins_per_degree, 1 / bins_per_degree)
    
    ax.set_title(title, fontsize=18)
    ax.hist(values, density=True, bins=bins, color="grey",
    edgecolor="1",
    linewidth=0.5)

    ax.set_xlabel('Angle [$\degree$]', fontsize=15, fontweight ='bold')
    ax.set_ylabel('Fraction', fontsize=15, fontweight ='bold')
    ax.tick_params(axis='x', labelsize=13)
    ax.tick_params(axis='y', labelsize=13)
#    ax.set_ylim([0, 0.2])
    mean = np.mean(values).item()
    std = np.std(values).item()
    ax.set_xlabel('Angle [$\degree$]')
    ax.set_ylabel('Fraction')
    if i != 0:
        ax.axvline(mean, c='k', ls='-', label=f'$\\bar{{x}}={round(mean if mean < 90 else -180 + mean, 2)}\degree$')
        ax.axvline(mean - std, c='k', ls='--', label=f'$s=\pm{round(std, 2)}\degree$')
        ax.axvline(mean + std, c='k', ls='--')
        plt.legend(prop={"size":14})

fig = plt.figure(figsize=(15, 10))
fig.suptitle('In-plane orientation', fontsize=22)
    
for i in tqdm(range(len(class_names) + 1)):
    ax = plt.subplot(2, 3, i + 1)
    hist(i, ax)
    
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

100%|██████████| 5/5 [00:03<00:00,  1.39it/s]


### Out-of-plane orientation (angle from xy-plane)
The histograms below show the distribution of the angle from the out-of-plane (XY-plane) angle.

In [27]:
def hist(i, ax):
    values = out_xy_angle[vec_class_masks[i]]
    values = np.array(values[~np.isnan(values)], dtype=np.float32)
    
    if i == 0:
        title = 'All classes'
        class_value = None
    else:
        title = f'Class {class_names[i - 1]}'
        class_value = int(class_names[i - 1])

    ax.set_title(title, fontsize=18)
    
    mean = np.mean(values).item()
    std = np.std(values).item()
    
    if class_value == 90:
        ax.set_xlim([-20, 20])
    else:
        ax.set_xlim([-10, 10])
    ax.hist(values, density=True, bins=np.arange(-90, 90 + 1 / bins_per_degree, 1 / bins_per_degree), color="grey",
    edgecolor="1",
    linewidth=0.75)
    ax.set_xlabel('Angle [$\degree$]', fontsize=15, fontweight ='bold')
    ax.set_ylabel('Fraction', fontsize=15, fontweight ='bold')
    ax.tick_params(axis='x', labelsize=13)
    ax.tick_params(axis='y', labelsize=13)
#    ax.set_ylim([0, 0.3])

#    ax.axvline(0, c='k', ls='--', label='$0\degree$')
    if i != 0:
        ax.axvline(mean, c='k', ls='-', label=f'$\\bar{{x}}={round(mean, 2)}\degree$')
        ax.axvline(mean - std, c='k', ls='--', label=f'$s=\pm{round(std, 2)}\degree$')
        ax.axvline(mean + std, c='k', ls='--')
        plt.legend(prop={"size":14})

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Out-of-plane orientation', fontsize=22)

for i in range(len(class_names) + 1):
    ax = plt.subplot(2, 3, i + 1)
    hist(i, ax)
    
plt.tight_layout()
plt.subplots_adjust(top=0.85)    
plt.show()

The class fractions and orientation distributions are useful for determining if the material follows production specifications. Here we've chosen a specific set of metrics to calculate and plot, but many other distributions can just as easily be obtained and plotted.

## Slices
As we chose to save the eigenvectors for each voxel in the ```data``` volume to the disk using the ```vec``` memory map, we can actually access all the eigenvectors along with the original data if we like. We also have orientation metrics avilable through the ```theta```, ```in_xy_angle```, ```out_xy_angle``` memory-maps, but they were saved with low precision intended only for visualization in other applications and we will not be using these data in this notebook.

Let's grab som data and their ST eigenvectors. The vectors were saved as ```float16``` to save space, but before we can plot them we have to change their type. Generally, CPUs do not support ```float16``` natively, so using this data type for anything but storing large data sets is generally not advisable.

**Note: We've chosen some specific slices from our data, such as Z-index 40 and 80. If you Z-dimension size is 80 or less this will give an error. You can easily change which slices will be shown in the figures below, by changing/adding/removing the ```data_slices``` and ```vec_slices``` indices.** 

In [28]:
data_slices = [
    data[data.shape[0] // 2].copy(),
    data[:, data.shape[1] // 2].copy(),
    data[:, :, data.shape[2] // 2].copy(),
    data[:, :, data.shape[2] // 2].copy(),
]

vec_slices = [
    vec[:, vec.shape[1] // 2].astype(np.float32),
    vec[:, :, vec.shape[2] // 2].astype(np.float32),
    vec[:, :, :, vec.shape[3] // 2].astype(np.float32),
    vec[:, :, :, vec.shape[3] // 3].astype(np.float32),
]

Let's can have a look at the data and the first element of the eigenvectors.

In [29]:
fig, axs = plt.subplots(1, len(data_slices), figsize=(20, 20))
for img, ax in zip(data_slices, axs):
    ax.imshow(img, cmap='gray')
plt.show()

fig, axs = plt.subplots(1, len(vec_slices), figsize=(20, 20))
for img, ax in zip(vec_slices, axs):
    ax.imshow(img[0])
plt.show()

## Slices with angle information
Just as we did in *StructureTensorFiberAnalysisDemo*, we can use the ```calculate_angles``` function to calculate the segmentation and orientation metrics for the slices we just read from the volumes.

First we define some plotting functions, then we calculate ```vec_class```, ```eta_os```, ```theta```, ```out_xy_angle```  and ```in_xy_angle``` for each of the slices and plot them. All the figures will be saved to the figures folder we created in the beginning. It easy to create figures for other slices simply be changing which slices are extracted from the volumes.

In [30]:
def add_scalebar(ax, length=100, scale=1, unit='μm', text=None):
    ax.set_xticks([])
    ax.set_yticks([])
    
    text = f'{length} {unit}' if text is None else text
    
    fontprops = fm.FontProperties(size=18)
    scalebar = AnchoredSizeBar(ax.transData,
                               length / scale, text, 'lower left', 
                               pad=0.3,
                               color='white',
                               frameon=False,
                               size_vertical=3,
                               fontproperties=fontprops)
    ax.add_artist(scalebar)
    
def savefig(ax, i):
    title = ax.get_title()
    ax.set_title(None)
    plt.savefig(os.path.join(fig_path, f'{title}_{i}.svg'), bbox_inches='tight', pad_inches=0, dpi=300)
    plt.savefig(os.path.join(fig_path, f'{title}_{i}.pdf'), bbox_inches='tight', pad_inches=0, dpi=300)
    ax.set_title(title)

**NOTE: You can easily change which figures are made by commenting out the calls to ```fig_with_colorbar``` in the ```show_metrics``` function.** 

In [31]:
def fig_with_colorbar(i, d, o, title, alpha=0.5, cmap=None, vmin=None, vmax=None):
    """Creates a figure with data, overlay and color bar."""
    o = np.rot90(o, k=-1)
    d = np.rot90(d, k=-1)
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='3%', pad=0.05)
    ax.imshow(d, cmap='gray')
    if np.issubdtype(o.dtype, np.integer):
        cmap = plt.get_cmap('gist_rainbow', len(class_names))
        im = ax.imshow(o, alpha=alpha, cmap=cmap, vmin=-.5, vmax=len(class_names) - .5)
        cbar = fig.colorbar(im, cax=cax, orientation='vertical', ticks=np.arange(len(class_names)))
        cbar.ax.set_yticklabels(class_names)
    else:
        im = ax.imshow(o, alpha=alpha, cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(im, cax=cax, orientation='vertical')
    ax.set_title(title)
    add_scalebar(ax, length=2000, scale=voxel_size, text='2 mm')
    if save_figures:
        savefig(ax, i)
    plt.show()

def show_metrics(data_slices, vec_slices, fiber_threshold=None):
    
    for i, (d, v) in enumerate(zip(data_slices, vec_slices)):
        vec_class, eta_os, theta, out_xy_angle, in_xy_angle = calculate_angles(v, class_vectors)

        if fiber_threshold is not None:
            fiber_mask = d <= fiber_threshold

            vec_class = np.ma.masked_where(fiber_mask, vec_class)          
            eta_os[fiber_mask] = np.nan
            theta[fiber_mask] = np.nan
            out_xy_angle[fiber_mask] = np.nan
            in_xy_angle[fiber_mask] = np.nan

        fig_with_colorbar(i, d, vec_class, 'Class', alpha=0.7)
        fig_with_colorbar(i, d, eta_os, '$\eta_O$(X)', alpha=0.5, vmin=0, vmax=1)
        fig_with_colorbar(i, d, theta, 'Angle from X (0-90 deg.)', alpha=0.5, vmin=0, vmax=90)
        fig_with_colorbar(i, d, in_xy_angle, 'In-plane orientation (-90-90 deg.)', cmap='bwr', alpha=0.5, vmin=-90, vmax=90)
        fig_with_colorbar(i, d, out_xy_angle, 'Out-of-plane orientation (-90-90 deg.)', cmap='bwr', alpha=0.5, vmin=-90, vmax=90)
        fig_with_colorbar(i, d, theta, 'Angle from X (0-10 deg.)', alpha=0.5, vmin=0, vmax=10)
        fig_with_colorbar(i, d, in_xy_angle, 'In-plane orientation (-10-10 deg.)', cmap='bwr', alpha=0.5, vmin=-10, vmax=10)
        fig_with_colorbar(i, d, out_xy_angle, 'Out-of-plane orientation (-10-10 deg.)', cmap='bwr', alpha=0.5, vmin=-10, vmax=10)
        
show_metrics(data_slices, vec_slices, fiber_threshold=fiber_threshold)

/tmp/ipykernel_41/2208991995.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1, figsize=(15, 15))


**********************

## Material orientation mapping

For the material orientation mapping for the integration point-wise option an Abaqus .inp-file must be supplied. The mapping output is supplied via a new .inp-file containing of the original .inp-file and the appened material orientation in an executable Abaqus syntax. Furthermore, element sets for Abaqus are output.

Opens the .inp-file from the simulation and finds the lines where parts can be removed

In [32]:
fn = '../data/bundle_X002.inp'
file1 = open(fn,"r") 


n_N_start=0
n_N_end=0
n_E_start=0
n_E_end=0
rest=0
for line in file1:
    n_N_start=n_N_start+1
    if "*NODE" in line:
        Start_Nodes=n_N_start+1
        n_N_end=n_N_start
        for line in file1:
            n_N_end=n_N_end+1
            if "**" in line:
                End_Nodes=n_N_end-1
                for line in file1:
                    n_E_start=n_E_start+1
                    if "*ELEMENT" in line:
                        Start_Elements=n_E_start+n_N_end+1
                        for line in file1:
                            n_E_end=n_E_end+1
                            if "**" in line:
                                End_Elements=n_E_end+n_E_start+n_N_end-1
                                for line in file1:
                                    rest=rest+1
                                break
                        break
                break
        break
SF_nodes=rest+n_E_end+n_E_start+n_N_end-End_Nodes
SF_elements=rest+n_E_end+n_E_start+n_N_end-End_Elements
print('Coordinate information of the nodes lies within lines', Start_Nodes, 'and', End_Nodes)
print('Element definition lies within lines', Start_Elements, 'and', End_Elements)

Coordinate information of the nodes lies within lines 34 and 135499
Element definition lies within lines 135504 and 791019


In [33]:
file_n1 = pd.read_csv(fn, skiprows=Start_Nodes-1, skipfooter=SF_nodes, header=None, engine='python')
file_e1 = pd.read_csv(fn, skiprows=Start_Elements-1, skipfooter=SF_elements, header=None, engine='python')
file_n1.to_csv('../results/File_nodes.csv',index=False)
file_n2= open('../results/File_nodes.csv', newline='')
file_e1.to_csv('../results/File_elements.csv',index=False)
file_e2= open('../results/File_elements.csv', newline='')

Creates a numpy array for all nodes and elements

In [34]:
# Creates a numpy array for the coordinates of every single node
#
nodes = csv.reader(file_n2)
header = next(nodes)

Max_no_Nd=np.shape(file_n1)[0]

nodes_arr=np.empty((Max_no_Nd,4))
for row in nodes:
    i=int(row[0])-1
    for j in range(4):
        if j==0:
            nodes_arr[i][j] = int(row[j])
        else:
            nodes_arr[i][j] = float(row[j])            
print('Node array created')
#
#
elements = csv.reader(file_e2)
header = next(elements)

No_Nodes_per_El=np.shape(file_e1)[1]-1
Max_no_El=np.shape(file_e1)[0]

elements_arr=np.empty((Max_no_El,No_Nodes_per_El+1))
for row in elements:
    k=int(row[0])-1
    for l in range(No_Nodes_per_El+1):
        elements_arr[k][l] = int(row[l])
elements_arr=elements_arr.astype(int)   
print('Element array created')

Node array created
Element array created


Here the position of the image data and finite element mesh get aligned. It is designed that the finite element mesh was based directly on the image data. If the finite element mesh is created independently adjustments might be necessary.

In [35]:
X0=start_X*voxel_size
Y0=start_Y*voxel_size
Z0=start_Z*voxel_size
print('X-origin:', X0)
print('Y-origin:', Y0)
print('Z-origin:', Z0)

X-origin: 0.0
Y-origin: 0.0
Z-origin: 0.0


# Mapping 

Here the mapping of the structure tensor outcome onto the single finite elements happens. Note that g3 and g1 are switched since the structure tensor vec has the order z, y, x. The data is output as an appendix to the original .inp-file in a new .inp-file. Additionally, element sets for the different bundle types based on the orientations are output.

In [39]:
# Mapping
#
#---------------------------------------------------------------------------
file1 = open("../data/bundle_X002.inp","r") 
output = []
for line in file1:
    if line.startswith('*SOLID SECTION'):
        output.append(f'{line[:-1]}' ', ORIENTATION=DISTRIBUTION_MAT_ORIENTATION_1'"\n")
    elif line.startswith('*Solid Section'):
        output.append(f'{line[:-1]}' ', ORIENTATION=DISTRIBUTION_MAT_ORIENTATION_1'"\n")
    else:
        output.append(line)
file1.close()
fn_new='../results/bundle_Y001_orient.inp'
f = open(fn_new, 'w')
f.writelines(output)
f.close()

el_uni = []
el_backing = []
el_biax = []
cogx_sum = np.empty(No_Nodes_per_El)
cogy_sum = np.empty(No_Nodes_per_El)
cogz_sum = np.empty(No_Nodes_per_El)
cogx     = np.empty(Max_no_El)
cogy     = np.empty(Max_no_El)
cogz     = np.empty(Max_no_El)
VecOrientX=np.empty(Max_no_El)
VecOrientY=np.empty(Max_no_El)
VecOrientZ=np.empty(Max_no_El)
with open(fn_new,'a') as output:
    output.write('**DISTRIBUTION_MAT_ORIENTATION' '\n')
    output.write('**' '\n')
    output.write('*DISTRIBUTION TABLE, NAME=TABLE_MAT_ORIENTATION_1' '\n')
    output.write('COORD3D, COORD3D,' '\n')
    output.write('*DISTRIBUTION, LOCATION=ELEMENT, TABLE=TABLE_MAT_ORIENTATION_1, NAME=DISTRIBUTION_MAT_ORIENTATION_1' '\n')
    output.write('         ,                       1.,                       0.,                       0.,                       0.,                       1.,                       0.' '\n')
    for m in range(Max_no_El):
        cogx_s=0
        cogy_s=0
        cogz_s=0
        for n in range(No_Nodes_per_El):
            cogx_sum[n]=nodes_arr[elements_arr[m,n+1]-1,1]+cogx_s
            cogy_sum[n]=nodes_arr[elements_arr[m,n+1]-1,2]+cogy_s
            cogz_sum[n]=nodes_arr[elements_arr[m,n+1]-1,3]+cogz_s
            cogx_s=cogx_sum[n]
            cogy_s=cogy_sum[n]
            cogz_s=cogz_sum[n]
        cogx[m]=cogx_s/No_Nodes_per_El
        cogy[m]=cogy_s/No_Nodes_per_El
        cogz[m]=cogz_s/No_Nodes_per_El

        g1=int((cogx[m]-X0)/voxel_size)
        g2=int((cogy[m]-Y0)/voxel_size)
        g3=int((cogz[m]-Z0)/voxel_size)
        if g1<vec.shape[3] and g1 >= 0 and g2<vec.shape[2] and g2 >= 0 and g3<vec.shape[1] and g3 >= 0:
                
                VecOrientX[m]=vec[2,g3,g2,g1]
                VecOrientY[m]=vec[1,g3,g2,g1]
                VecOrientZ[m]=vec[0,g3,g2,g1]
                
        else:
                VecOrientX[m]=1
                VecOrientY[m]=0
                VecOrientZ[m]=0
    m=0
    for m in range(Max_no_El):
        output.write(f'{m+1},        {VecOrientZ[m]:.4f},        {VecOrientY[m]:.4f},        {VecOrientX[m]:.4f},                       1.,                       0.,                       0.' '\n')

        if abs(VecOrientX[m]) > 0.9:
            el_uni.append(m+1)
        elif abs(VecOrientX[m]) < 0.1 and abs(VecOrientX[m]) >0.00000001:
            el_backing.append(m+1)
        elif abs(VecOrientX[m]) > 0.1 and abs(VecOrientX[m]) < 0.9:
            el_biax.append(m+1)
#       
#            
    output.write('*ORIENTATION, NAME=DISTRIBUTION_MAT_ORIENTATION_1, DEFINITION=COORDINATES, SYSTEM=RECTANGULAR' '\n')
    output.write('DISTRIBUTION_MAT_ORIENTATION_1,' '\n')
    output.write('       3,                       0.' '\n')
    output.write('*ELSET, ELSET=El_UNI' '\n')
    l=0
    n=0
    p=0
    k=1
    for l in range(len(el_uni)):
        
        if k==10:
            output.write(f' {el_uni[l]},' '\n')
            k=1
        else:
            output.write(f' {el_uni[l]},') 
            k=k+1
            
    output.write('\n')
    
    output.write('*ELSET, ELSET=El_Backing' '\n')
    m=1
    for n in range(len(el_backing)):
        
        if m==10:
            output.write(f' {el_backing[n]},' '\n')
            m=1
        else:
            output.write(f' {el_backing[n]},') 
            m=m+1

    output.write('\n')
            
    output.write('*ELSET, ELSET=El_Biax' '\n')
    o=1
    for p in range(len(el_biax)):
        
        if o==10:
            output.write(f' {el_biax[p]},' '\n')
            o=1
        else:
            output.write(f' {el_biax[p]},') 
            o=o+1
#